### Fine-tuning TinyBERT for Sentiment Analysis

#### Introduction and Objective

The purpose of this notebook is to develop a sentiment analysis model using the TinyBERT architecture. Sentiment analysis, a key task in natural language processing (NLP), involves determining the sentiment expressed in a piece of text, such as positive, negative, or neutral. By leveraging the lightweight and efficient TinyBERT model, we aim to achieve accurate sentiment classification on a dataset of tweets. This analysis can provide valuable insights into public opinion and sentiment trends, which are crucial for various applications such as market analysis, customer feedback, and social media monitoring.

#### Required Datasets

Sentiment Analysis Data - [Kaggle Dataset](https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset?select=test.csv)

#### Dataset Overview

The dataset used in this project is sourced from Kaggle and contains text data from tweets, labeled with sentiment categories. The dataset provides a rich source of real-world text data that can be used to train and evaluate the TinyBERT model for sentiment analysis. Key features of the dataset include:

- **textID**: A unique identifier for each tweet in the dataset.
- **text**: The actual tweet content, which is the primary input for sentiment analysis.
- **selected_text**: The portion of the tweet that reflects the sentiment.
- **sentiment**: The label indicating the sentiment of the tweet, typically categorized as positive, negative, or neutral.
- **Time of Tweet**: The time when the tweet was posted, categorized into different periods of the day (morning, noon, night).
- **Age of User**: The age group of the user who posted the tweet.
- **Country**: The country from which the tweet was posted.
- **Population -2020**: The population of the country as of the year 2020.
- **Land Area (Km²)**: The land area of the country in square kilometers.
- **Density (P/Km²)**: The population density of the country per square kilometer.

#### Features Used

For this project, we will focus only on the **text** and **sentiment** fields, as these are directly relevant to our goal of sentiment classification. The **text** field serves as the input text that the TinyBERT model will analyze, and the **sentiment** field is the target variable for the classification task.

**Install Required Packages**:

   To enhance the functionality of the environment, you may need to install some libraries not pre-installed in CoreAI environment but required for this notebook. Follow these steps to install the necessary libraries from the `requirements.txt` file:

   **1 Create and Activate the Virtual Environment:**
   
   Open your terminal or command prompt within the Jupyter notebook. `File -> New -> Terminal` and type `bash` to get a shell compatible with the following commands.
   
   Navigate to the project directory where the notebook is to set up the environment.
   
   Execute the following commands to create and activate the virtual environment:
   
   ```
   python3 -m venv --system-site-packages myvenv
   source myvenv/bin/activate
   pip3 install ipykernel
   python -m ipykernel install --user --name=myvenv --display-name="Python (myvenv)"
   ```

   **2 Install Required Libraries**
   
   Before running the following command in Jupyter notebook, make sure you are in the directory where the Jupyter Notebook and virtual environment is located. Load the newly created "Python (myenv)" kernel. This ensures the `./` path is always current. You can use the `cd` command to change to your project directory and `pwd` to verify your current directory.

In [ ]:
!. ./myvenv/bin/activate; pip install -r requirements.txt

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, IterableDataset
from transformers import BertTokenizerFast, BertForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from torch.cuda.amp import autocast, GradScaler
import os
import json
import matplotlib.pyplot as plt
from wordcloud import WordCloud
# Download NLTK resources within the myvenv (local) which is inside the NLTK default search path
nltk.download('punkt', download_dir='./myvenv/nltk_data')
nltk.download('stopwords', download_dir='./myvenv/nltk_data')
nltk.download('wordnet', download_dir='./myvenv/nltk_data')

# Data Exploration

In [ ]:
train_data = pd.read_csv('data/train.csv', encoding='latin1')
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.isna().sum()

In [ ]:
train_data.dropna(inplace=True)

In [ ]:
train_data.duplicated().sum()

In [ ]:
train_data['sentiment'].unique()

In [ ]:
train_data['sentiment'].value_counts()

# Exploratory Data Analysis

In [ ]:
plt.pie(train_data['sentiment'].value_counts(),labels=['Neutral','Positive','Negative'], autopct='%1.2f%%',
            colors=['#008083','#FF7F50','#FFFF96'],wedgeprops=dict(width=0.7))
plt.show()

- The dataset for sentiment analysis is fairly balanced, with 40.45% of the tweets labeled as neutral, 31.23% as positive, and 28.32% as negative. This distribution indicates that the model will have a relatively balanced amount of data for each sentiment category, which can help in achieving a more generalized and accurate model.

In [ ]:
positive_data=train_data[train_data['sentiment']=='positive']

In [ ]:
text = ' '.join(positive_data['text'].astype(str))


stop_words = set(stopwords.words('english'))
# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stop_words).generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Turn off the axis
plt.title("Positive Sentiment's Word Cloud \n",fontsize=24,color='grey')
plt.axis('off')  # Turn off the axis
plt.show()

- The word cloud for positive sentiments shows that the most frequently occurring words in positive tweets include "love," "thank," "happy," "great," and "day." These words reflect common expressions of positive emotions and sentiments in the dataset, indicating that the model can focus on these kind of key terms to effectively identify positive sentiment in tweets.

In [ ]:
negative_data=train_data[train_data['sentiment']=='negative']
text = ' '.join(negative_data['text'].astype(str))
stop_words = set(stopwords.words('english'))
# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stop_words).generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title("Negative Sentiment's Word Cloud \n",fontsize=24,color='grey')
plt.axis('off')  # Turn off the axis
plt.show()

- The word cloud for negative sentiments shows that the most frequently occurring words in negative tweets include "work," "get," "hate," "miss," and "going." These words reflect common expressions of negative emotions and sentiments in the dataset, indicating that the model can focus on these key terms to effectively identify negative sentiment in tweets.

In [ ]:
neutral_data=train_data[train_data['sentiment']=='neutral']
text = ' '.join(neutral_data['text'].astype(str))
stop_words = set(stopwords.words('english'))
# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stop_words).generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title("Neutral Sentiment's Word Cloud \n",fontsize=24,color='grey')
plt.axis('off')  # Turn off the axis
plt.show()

- The word cloud for neutral sentiments shows that the most frequently occurring words in neutral tweets include "day," "get," "going," "work," and "today." These words reflect common expressions of everyday activities and neutral statements, indicating that the model can focus on these key terms to effectively identify neutral sentiment in tweets.

In [ ]:
train_data['Time of Tweet'].value_counts()

In [ ]:
grouped_data = train_data.groupby(['Time of Tweet', 'sentiment']).size().reset_index(name='count')
pivot_data = grouped_data.pivot(index="Time of Tweet", columns="sentiment", values="count").fillna(0)
fig, ax = plt.subplots(figsize=(10, 6))
cax = ax.matshow(pivot_data, cmap='Reds')
fig.colorbar(cax)
ax.set_xticks(np.arange(len(pivot_data.columns)))
ax.set_yticks(np.arange(len(pivot_data.index)))
ax.set_xticklabels(pivot_data.columns)
ax.set_yticklabels(pivot_data.index)
for (i, j), val in np.ndenumerate(pivot_data.values):
    ax.text(j, i, int(val), ha='center', va='center', color='black')
plt.xlabel('Sentiment')
plt.ylabel('Time of Tweet')
plt.title('Heat Map of Tweet Times and Sentiments \n\n')
plt.show()

- The heatmap shows that tweets with neutral sentiments are the most frequent across all times of the day (morning, noon, night), with the highest concentration occurring at night. Positive sentiments are more common in the morning and noon, while negative sentiments are relatively evenly distributed throughout the day. This indicates a general trend of higher neutral sentiment activity in the dataset, particularly during nighttime hours.

In [ ]:
grouped_data = train_data.groupby(['Age of User', 'sentiment']).size().reset_index(name='count')
pivot_data = grouped_data.pivot(index='Age of User', columns='sentiment', values='count').fillna(0)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 18))

# Define bar width
bar_width = 0.9

# Plot each sentiment separately
sentiments = ['negative', 'neutral', 'positive']
colors = ['#FFF796', '#008083', '#FF7F50']

for ax, sentiment, color in zip(axes, sentiments, colors):
    ax.bar(pivot_data.index, pivot_data[sentiment], bar_width, color=color, label=sentiment.capitalize())
    ax.set_title(f'{sentiment.capitalize()} Sentiment Distribution by Age Group')
    ax.set_xlabel('Age of User')
    ax.set_ylabel('Count')
    ax.set_xticks(np.arange(len(pivot_data.index)))
    ax.set_xticklabels(pivot_data.index, rotation=0)
    ax.legend(title='Sentiment')

# Adjust layout
plt.tight_layout()
plt.show()

- The distribution of negative sentiments is fairly uniform across all age groups, indicating that negative sentiments are consistently expressed by users of all ages.

- The distribution of neutral sentiments is also relatively uniform across all age groups, but the counts are slightly higher compared to negative sentiments, suggesting that users tend to express neutral sentiments more frequently.

- Positive sentiments are uniformly distributed across all age groups, similar to the negative sentiment distribution, with a slightly higher count, indicating a balanced expression of positive sentiments among users of different ages.

In [ ]:
from collections import Counter

### Model Building
In this section, we will fine-tune the TinyBERT model for sentiment analysis using a custom dataset of tweets. The steps involved include loading and preprocessing the dataset, creating data loaders for training and validation, defining the model and optimizer, and implementing the training and evaluation loops. Finally, we will save the trained model, tokenizer, and evaluation metrics for future use.

In [ ]:
import nltk
nltk.download('punkt_tab')

class TextIterableDataset(IterableDataset):
    """
    A custom iterable dataset for text classification.

    Args:
        texts (list): A list of input texts.
        labels (list): A list of corresponding labels.

    Attributes:
        texts (list): A list of input texts.
        labels (list): A list of corresponding labels.
        tokenizer (BertTokenizerFast): The tokenizer used for text preprocessing.

    Methods:
        preprocess_text(text): Preprocesses the input text by tokenizing, removing stopwords, and lemmatizing.
        process_data(text, label): Preprocesses the input text and returns the processed data as a dictionary.
        __iter__(): Iterates over the texts and labels, yielding the processed data for each pair.

    """

    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

    def preprocess_text(self, text):
        """
        Preprocesses the input text by tokenizing, removing stopwords, and lemmatizing.

        Args:
            text (str): The input text to be preprocessed.

        Returns:
            str: The preprocessed text.

        """
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        cleaned_text = ' '.join([lemmatizer.lemmatize(w.lower()) for w in tokens if w.isalpha() and w not in stop_words])
        return cleaned_text

    def process_data(self, text, label):
        """
        Preprocesses the input text and returns the processed data as a dictionary.

        Args:
            text (str): The input text to be preprocessed.
            label (int): The corresponding label for the input text.

        Returns:
            dict: The processed data as a dictionary with keys 'input_ids', 'attention_mask', and 'labels'.

        """
        text = self.preprocess_text(text)
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        return {'input_ids': encoding['input_ids'].squeeze(0), 'attention_mask': encoding['attention_mask'].squeeze(0), 'labels': torch.tensor(label)}

    def __iter__(self):
        """
        Iterates over the texts and labels, yielding the processed data for each pair.

        Yields:
            dict: The processed data as a dictionary with keys 'input_ids', 'attention_mask', and 'labels'.

        """
        for text, label in zip(self.texts, self.labels):
            yield self.process_data(text, label)

def collate_fn(batch):
    """
    Collate function for batching data.

    Args:
        batch (list): A list of dictionaries, where each dictionary represents a single data sample.

    Returns:
        dict: A dictionary containing the batched data with keys 'input_ids', 'attention_mask', and 'labels'.
    """
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

def load_dataset(file_path):
    """
    Loads the dataset from a CSV file.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        tuple: A tuple containing the list of texts and the list of sentiments.

    """
    try:
        df = pd.read_csv(file_path, dtype=str, low_memory=False, encoding='latin1')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, dtype=str, low_memory=False, encoding='ISO-8859-1')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, dtype=str, low_memory=False, encoding='latin1')
    
    df.dropna(subset=['text', 'sentiment'], inplace=True)
    sentiment_mapping = {'neutral': 0, 'negative': 1, 'positive': 2}
    df['sentiment'] = df['sentiment'].map(sentiment_mapping)    
    df.dropna(subset=['sentiment'], inplace=True)
    df['sentiment'] = df['sentiment'].astype(int)
    return df['text'].tolist(), df['sentiment'].tolist()

def train(model, train_loader, optimizer, device, scaler):
    """
    Trains the model on the training data.

    Args:
        model (BertForSequenceClassification): The model to be trained.
        train_loader (DataLoader): The data loader for the training data.
        optimizer (AdamW): The optimizer used for training.
        device (torch.device): The device to be used for training.
        scaler (GradScaler): The gradient scaler for mixed precision training.

    """
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

def evaluate(model, data_loader, device):
    """
    Evaluates the model on the validation or test data.

    Args:
        model (BertForSequenceClassification): The model to be evaluated.
        data_loader (DataLoader): The data loader for the validation or test data.
        device (torch.device): The device to be used for evaluation.

    Returns:
        tuple: A tuple containing the accuracy, precision, recall, and F1 score.

    """
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions.extend(torch.argmax(outputs.logits, -1).tolist())
            true_labels.extend(labels.tolist())
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
    return accuracy, precision, recall, f1

def save_model_and_tokenizer(model, tokenizer, model_path, tokenizer_path):
    """
    Saves the model and tokenizer to the specified paths.

    Args:
        model (BertForSequenceClassification): The model to be saved.
        tokenizer (BertTokenizerFast): The tokenizer to be saved.
        model_path (str): The path to save the model.
        tokenizer_path (str): The path to save the tokenizer.

    """
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    model.save_pretrained(model_path)
    if not os.path.exists(tokenizer_path):
        os.makedirs(tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)

def save_test_data_as_csv(test_texts, test_labels, file_path):
    """
    Saves the test data as a CSV file.

    Args:
        test_texts (list): The list of test texts.
        test_labels (list): The list of test labels.
        file_path (str): The path to save the CSV file.

    """
    df_test = pd.DataFrame({'text': test_texts, 'label': test_labels})
    df_test.to_csv(f"{file_path}/test_dataset.csv", index=False)

def save_evaluation_metrics_as_json(metrics, file_path):
    """
    Saves the evaluation metrics as a JSON file.

    Args:
        metrics (tuple): A tuple containing the evaluation metrics.
        file_path (str): The path to save the JSON file.

    """
    with open(f"{file_path}/evaluation_metrics.json", 'w') as f:
        json.dump(metrics, f)

# Main function
if __name__ == "__main__":
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load dataset
    file_path = 'data/train.csv'
    texts, labels = load_dataset(file_path)

    # Split dataset
    train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
    val_texts, test_texts, val_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=42)

    # Create datasets and data loaders
    train_dataset = TextIterableDataset(train_texts, train_labels)
    val_dataset = TextIterableDataset(val_texts, val_labels)
    test_dataset = TextIterableDataset(test_texts, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=16, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

    # Train and evaluate model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    scaler = GradScaler()

    for epoch in range(3):
        train(model, train_loader, optimizer, device, scaler)
        accuracy, precision, recall, f1 = evaluate(model, val_loader, device)
        print(f'Epoch {epoch+1}, Val Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

    metrics = evaluate(model, test_loader, device)
    print(f'Test Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}')

    # Save model, tokenizer, test data, and evaluation metrics
    model_save_path = 'Sentiment_Analysis'
    save_model_and_tokenizer(model, BertTokenizerFast.from_pretrained('bert-base-uncased'), model_path=model_save_path, tokenizer_path=model_save_path)
    save_test_data_as_csv(test_texts, test_labels, model_save_path)
    save_evaluation_metrics_as_json(metrics, model_save_path)

In [ ]:

# Import libraries
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification
import numpy as np

def preprocess_text(text, tokenizer, max_len=512):
    """
    Preprocesses the input text by tokenizing it using the provided tokenizer.

    Args:
        text (str): The input text to be preprocessed.
        tokenizer (Tokenizer): The tokenizer object used for tokenization.
        max_len (int, optional): The maximum length of the tokenized sequence. Defaults to 512.

    Returns:
        tokens (Tensor): The tokenized and preprocessed text as a PyTorch tensor.
    """
    tokens = tokenizer(text, padding='max_length', truncation=True, max_length=max_len, return_tensors="pt")
    return tokens

def predict_sentiment(text, model, tokenizer):
    """
    Predicts the sentiment of the given text using the provided model and tokenizer.

    Args:
        text (str): The input text to analyze.
        model: The pre-trained sentiment analysis model.
        tokenizer: The tokenizer used to preprocess the text.

    Returns:
        predicted_class (numpy.ndarray): The predicted sentiment class.
        confidence (numpy.ndarray): The confidence scores for each sentiment class.
        probabilities (numpy.ndarray): The probability distribution over all sentiment classes.
    """
    preprocessed_text = preprocess_text(text, tokenizer)
    input_ids = preprocessed_text['input_ids']
    attention_mask = preprocessed_text['attention_mask']

    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        confidence, predicted_class = torch.max(probabilities, dim=-1)

    return predicted_class.cpu().numpy(), confidence.cpu().numpy(), probabilities.cpu().numpy()

# Load the pre-trained sentiment analysis model and tokenizer
model_path = 'Sentiment_Analysis'
model = BertForSequenceClassification.from_pretrained(model_path).to('cuda')
tokenizer = BertTokenizerFast.from_pretrained(model_path)

# Perform sentiment analysis on user input
while True:
    # Get user input
    text = input("\nEnter text for sentiment analysis (or type 'exit' to stop): ")

    # Check if the user wants to exit
    if text.lower() == 'exit':
        print("Exiting sentiment analysis.")
        break

    # Predict the sentiment of the input text
    predicted_class, confidence, probabilities = predict_sentiment(text, model, tokenizer)
    sentiment_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

    # Display the predicted sentiment and confidence
    print(f"\nSentiment: {sentiment_labels[predicted_class.item()]}")
    print(f"Confidence: {confidence.item() * 100:.2f}%")

    # Display the probability distribution over all sentiment classes
    print("Probabilities:")
    for sentiment, probability in zip(sentiment_labels.values(), probabilities[0]):
        print(f"  {sentiment}: {probability.item() * 100:.2f}%")



Sentiment: Positive
Confidence: 82.09%
Probabilities:
  Negative: 8.79%
  Neutral: 9.12%
  Positive: 82.09%



Enter text for sentiment analysis (or type 'exit' to stop):  I don't like you



Sentiment: Neutral
Confidence: 98.35%
Probabilities:
  Negative: 1.21%
  Neutral: 98.35%
  Positive: 0.44%



Enter text for sentiment analysis (or type 'exit' to stop):  exit


Exiting sentiment analysis.
